### Ground truth dataset persistence and evaluation in TruLens

In this notebook, we give a quick walkthrough of how you can prepare your own ground truth dataset, as well as utilize our utility function to load preprocessed BEIR (Benchmarking IR) datasets to take advantage of its unified format.


In [ ]:
# !pip install trulens trulens-provider-openai openai

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "sk-..."

In [ ]:
import pandas as pd
from trulens.core import Tru

tru = Tru()
tru.reset_database()

In [ ]:
data = {
    "query": ["hello world", "who is the president?", "what is AI?"],
    "query_id": ["1", "2", "3"],
    "expected_response": ["greeting", "Joe Biden", "Artificial Intelligence"],
    "expected_chunks": [
        None,
        None,
        None,
    ],  # This can also be lists if applicable
    "meta": [None, None, None],  # Metadata can also be a list of dictionaries
}

df = pd.DataFrame(data)

### Idempotency:
 IDs for both datasets and ground truth data entries are based on their content and metadata, so `add_ground_truth_to_dataset` is idempotent and should not create duplicate rows in the DB. 

In [ ]:
tru.add_ground_truth_to_dataset(
    dataset_name="test_dataset_new", ground_truth_df=df
)

### Load a preprocessed dataset from BEIR (Benchmarking Information Retrieval) collection

In [ ]:
from trulens.core.dataset.beir_loader import TruBEIRDataLoader

beir_data_loader = TruBEIRDataLoader()
df = beir_data_loader.load_dataset_to_df(dataset_name="scifact", data_path="./")

In [ ]:
df